<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_Deep_Learning_Con_TernsorFlow_Machine-Learning_e_IA-/blob/master/scripts/tema11/Clase_11_taller_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Divide el data frame en conjunto de entrenamiento y test para aplicar diversas técnicas de las aprendidas en el curso.

In [ ]:
pip install tensorflow==1.15

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import csv
print(tf.__version__)

In [ ]:
session = tf.Session()
csv_file = 'original.csv'

scores = []

with open(csv_file,'rt') as csvfile:
    reader = csv.reader(csvfile, delimiter =',')
    for row in reader:
      scores.append(([x.replace('Female', '0').replace('Male', '1') for x in row[0:5]]))

csv_scores = scores
scores_header = csv_scores[0]

In [ ]:
csv_scores[0:6]

In [ ]:
scores_header

In [ ]:
y_vals = np.array([int(x[4]) for x in csv_scores[1:] if len(x)>=1])
x_vals = np.array([list(map(int, x[0:4])) for x in csv_scores[1:] if len(x)>=1])

seed = 2020
tf.set_random_seed(seed)
np.random.seed(seed)
batch_size = 20

train_idx = np.random.choice(len(x_vals), round(len(x_vals)*0.8), replace=False)
test_idx = np.array(list(set(range(len(x_vals)))-set(train_idx)))

x_vals_train = x_vals[train_idx]
x_vals_test = x_vals[test_idx]
y_vals_train = y_vals[train_idx]
y_vals_test = y_vals[test_idx]

In [ ]:
y_vals_test

In [ ]:
def normalize_cols(m, col_min = np.array([None]), col_max = np.array([None])):
  if not col_min[0]:
      col_min = m.min(axis=0)
  if not col_max[0]:
      col_max = m.max(axis=0)
  return(m-col_min)/(col_max-col_min), col_min, col_max

x_vals_train, train_min, train_max = np.nan_to_num(normalize_cols(x_vals_train))
x_vals_test,_,_ = np.nan_to_num(normalize_cols(x_vals_test, train_min, train_max))

In [ ]:
x_vals_test

## 2. Utiliza las diferentes técnicas vistas durante el curso para predecir el Spending Score del cliente en base al resto de columnas del data frame. 

In [ ]:
def init_weight(shape, st_dev):
  weight = tf.Variable(tf.random_normal(shape = shape, stddev=st_dev))
  return weight

def init_bias(shape, st_dev):
  bias = tf.Variable(tf.random_normal(shape = shape, stddev=st_dev))
  return bias

x_data = tf.placeholder(shape=[None, 4], dtype = tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

def full_connected(input_layer, weights, biases):
  layer = tf.add(tf.matmul(input_layer, weights), biases)
  return(layer)

w1 = init_weight(shape=[4,25], st_dev=10.0)
b1 = init_bias(shape=[25], st_dev=10.0)
layer1 = full_connected(x_data, w1, b1)

w2 = init_weight(shape=[25,10], st_dev=10.0)
b2 = init_bias(shape=[10], st_dev=10.0)
layer2 = full_connected(layer1, w2, b2)

w3 = init_weight(shape=[10,3], st_dev=10.0)
b3 = init_bias(shape=[3], st_dev=10.0)
layer3 = full_connected(layer2, w3, b3)

w4 = init_weight(shape=[3,1], st_dev=10.0)
b4 = init_bias(shape=[1], st_dev=10.0)
layer4 = full_connected(layer3, w4, b4)

oss_vects = []

test_loss = []

for i in range(300):
  rand_idx = np.random.choice(len(x_vals_train), size = batch_size)
  rand_x = x_vals_train[rand_idx]
  rand_y = np.transpose([y_vals_train[rand_idx]])  

  session.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y}) 
  temp_loss = session.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
  loss_vects.append(temp_loss)
  

  test_temp_loss = session.run(loss, feed_dict={x_data: x_vals_test, y_target: np.transpose([y_vals_test])})
  test_loss.append(test_temp_loss)  

  if(i+1)%25==0:

      print("Paso #"+str(i+1)+", Loss = "+str(temp_loss))

## 3. Aplica la validación cruzada para comprobar que el conjunto de test se comporta bien y que tu algoritmo hace una buena predicción.

In [ ]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model_output, labels = targets))
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)
prediction = tf.argmax(model_output,1)